Support Vector Classifiers


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# ---------------- 读取数据 ----------------
file_name = "D:/MyProject/MatContest/dataset/raw/yf2014_2018.csv" # 定义要读取的 CSV 文件路径
dataset = pd.read_csv(file_name)
dataset.head()  # 显示前五行数据

,Close,High,Low,Open,Volume
0,3.95,3.98,3.84,3.85,20548400
1,4.00,4.00,3.88,3.98,22887200
2,4.13,4.18,3.99,4.01,42398300
3,4.18,4.25,4.11,4.19,42932100
4,4.18,4.26,4.14,4.23,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Close'].shift(-1) > dataset['Close'],1,0)
dataset['Returns'] = dataset['Close'].pct_change()
dataset = dataset.dropna()

In [4]:
dataset.head()

,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
1,4.00,4.00,3.88,3.98,22887200,1,1,1,0.012658
2,4.13,4.18,3.99,4.01,42398300,1,1,1,0.032500
3,4.18,4.25,4.11,4.19,42932100,0,1,0,0.012106
4,4.18,4.26,4.14,4.23,30678700,0,0,0,0.000000
5,4.09,4.23,4.05,4.20,30667600,0,0,1,-0.021531


In [5]:
dataset.tail()

,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
1166,19.980000,20.08,19.350000,19.790001,62983200,0,1,1,0.010622
1167,20.400000,20.42,19.860001,19.980000,55629000,1,1,1,0.021021
1168,20.900000,20.92,20.209999,20.280001,62002700,1,1,1,0.024510
1169,22.290001,22.32,21.139999,21.190001,113444100,1,1,1,0.066507
1170,23.980000,24.00,22.670000,22.910000,164328200,0,0,0,0.075819


In [6]:
dataset.shape

(1170, 9)

In [7]:
X = dataset.drop(['Buy_Sell'], axis=1).values
y = dataset['Buy_Sell'].values

In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (877, 8) (877,)
Test set: (293, 8) (293,)


In [27]:
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
model = SVC(kernel = 'rbf', C = 1000,gamma=0.001)
model.fit(X_train, y_train)
svc_predictions = model.predict(X_test)

In [10]:
print("Accuracy of SVM using optimized parameters ", accuracy_score(y_test, svc_predictions) * 100)
print("Report : ", classification_report(y_test, svc_predictions))

Accuracy of SVM using optimized parameters  48.80546075085324
Report :                precision    recall  f1-score   support

           0       0.49      0.99      0.66       144
           1       0.00      0.00      0.00       149

    accuracy                           0.49       293
   macro avg       0.24      0.50      0.33       293
weighted avg       0.24      0.49      0.32       293



In [11]:
model.score(X_test, y_test)

0.4880546075085324

Support vector Machines

总结整个流程

1. 读取并初步查看数据。
2. 构建目标列（未来涨跌）和数值特征（用 np.where）。
3. 提取特征矩阵 X 和目标向量 y。
4. 标准化特征以适应 SVM。
5. 划分训练集和测试集。
6. 训练 SVM 分类模型。
7. 对测试集进行预测并计算准确率。
8. 支持对单个新样本进行预测。


In [12]:
import numpy as np
import pandas as pd
# ---------------- 读取数据 ----------------
file_name = "D:/MyProject/MatContest/dataset/raw/yf2014_2018.csv" # 定义要读取的 CSV 文件路径
dataset1 = pd.read_csv(file_name)
dataset1.head()  # 显示前五行数据

,Close,High,Low,Open,Volume
0,3.95,3.98,3.84,3.85,20548400
1,4.00,4.00,3.88,3.98,22887200
2,4.13,4.18,3.99,4.01,42398300
3,4.18,4.25,4.11,4.19,42932100
4,4.18,4.26,4.14,4.23,30678700


In [26]:
# 构建目标列 Up/Down
dataset1['Return'] = dataset1['Close'].pct_change() # 计算每日收益率
# 创建目标列 Up_Down：明天收益率是否大于今天
dataset1['Up_Down'] = np.where(dataset1['Return'].shift(-1) > dataset1['Return'], 'Up', 'Down')
dataset1 = dataset1.dropna()

In [25]:
# 构建特征列并直接用 np.where 转为数值
dataset1['Open_N_num'] = np.where(dataset1['Open'].shift(-1) > dataset1['Open'], 1, 0)       # Up=1, Down=0
dataset1['Volume_N_num'] = np.where(dataset1['Volume'].shift(-1) > dataset1['Volume'], 1, 0) # Positive=1, Negative=0

In [29]:
# 提取特征和目标
# 选择特征列 Open, Open_N_num, Volume_N_num 作为模型输入。
# 选择目标列 Up_Down 作为模型输出。
X = dataset1[['Open', 'Open_N_num', 'Volume_N_num']].values
y = dataset1['Up_Down'].values

In [31]:
# 特征标准化
# 将每列特征标准化为均值 0、标准差 1。
# 避免 SVM 被特征数值尺度影响，提高训练稳定性
'''
SVM 的核心是求最大间隔超平面（Hyperplane）：
决策边界取决于 特征空间中的距离（欧氏距离或核函数计算的距离）。
如果不同特征的数值尺度差异很大，距离会被大数值特征主导，导致模型不合理

例子：
特征 A 范围在 [0, 1]
特征 B 范围在 [0, 1000]
SVM 会几乎只关注特征 B，忽略特征 A 的信息

'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [33]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale')  # RBF 核
svm_classifier.fit(X_train, y_train) # 用训练集训练模型，学习输入特征和 'Up'/'Down' 的映射关系

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [35]:
y_pred = svm_classifier.predict(X_test)

In [36]:
from sklearn import metrics
print("SVM Accuracy:", metrics.accuracy_score(y_test, y_pred))

SVM Accuracy: 0.6239316239316239


In [38]:
# 手动计算准确率
boolian = (y_test == y_pred)
accuracy_manual = sum(boolian) / y_test.size
print("Manual Accuracy:", accuracy_manual)

Manual Accuracy: 0.6239316239316239


In [40]:
# 定义一个新的输入样本（特征已用数值表示）
data_in = [4.0, 1, 1]  # 示例：Open=4.0, Open_N_num=1 (Up), Volume_N_num=1 (Positive)
# 标准化特征，使其与训练数据同一尺度
data_in_scaled = scaler.transform([data_in])
# 调用 predict 预测该样本属于 'Up' 还是 'Down'
yhat = svm_classifier.predict(data_in_scaled)
print("Prediction for input sample:", yhat[0])

Prediction for input sample: Down
